In [82]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score

pd.set_option('display.max_columns', 50)

In [2]:
def metrics(y_true, y_pred):
    '''
    Calculates binary classification performance metrics for a given model.
    :param y_true: array_like, truth values as int
    :param y_pred: array_like, predicted values as int
    :returns: dict, with keys for each metric: 
        accuracy - proportion of correct predictions out of total predictions
        sensitivity - (aka recall), of all true positives reviews how many did we correctly predict as positive
        specificity - (aka selectivity/TNR), of all true negatives how many did we correctly predict as negative
        precision - of all predicted positive cases how many were actually positive
        F-1 score - harmonic/weighted mean of precision and sensitivity scores
        ROC-AUC - area under receiver operating characteristic curve
        
    '''
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    metrics_dict = {}
    metrics_dict['accuracy'] = round((tp + tn) / len(y_true), 4)
    metrics_dict['sensitivity/recall'] = round(tp / (fn + tp), 4) # aka recall
    metrics_dict['specificity'] = round(tn / (tn + fp), 4) # aka TNR
    metrics_dict['precision'] = round(tp / (tp + fp), 4)
    metrics_dict['f1'] = round(2 * (metrics_dict['precision'] * metrics_dict['sensitivity/recall']) \
                        / (metrics_dict['precision'] + metrics_dict['sensitivity/recall']), 4)
    metrics_dict['roc_auc'] = round(roc_auc_score(y_true, y_pred), 4)
    
    return metrics_dict

In [3]:
def combine_prediction(X_test, pred, outcome='50k'):
    X_test_combined = pd.concat([X_test, pd.DataFrame(y_test, columns=[outcome])], axis=1)
    return pd.concat([X_test_combined.reset_index(drop=True), pd.DataFrame(pred, columns=['pred'])], axis=1)

# Data preparation

### Read in [2019 ASEC (March CPS)](https://cps.ipums.org/cps/)

In [4]:
df = pd.read_csv('./data/asec_2019.csv')

In [5]:
print(df.shape)

(180101, 18)


In [6]:
print(df.head())

   YEAR  MONTH  ASECFLAG   ASECWT  RELATE  AGE  SEX  RACE  MARST  POPSTAT  \
0  2019      3         1  2031.67     101   21    1   100      6        1   
1  2019      3         1  1232.04     101   85    2   100      5        1   
2  2019      3         1  1209.17     101   61    2   100      6        1   
3  2019      3         1  1146.23     101   73    2   100      4        1   
4  2019      3         1  1480.79     301   37    1   100      6        1   

   HISPAN  LABFORCE  UHRSWORK1  EDUC  SCHLCOLL  WKSWORK1  UHRSWORKLY  INCWAGE  
0       0         2       30.0    60       5.0        52          30  18000.0  
1       0         1      999.0    73       0.0         0         999      0.0  
2       0         2       44.0    73       0.0        52          44  12000.0  
3       0         1      999.0    73       0.0         0         999      0.0  
4       0         2       20.0    73       5.0        52          20  12000.0  


### Restrict dataset

In [7]:
# Drop individuals with no employment or earnings last year
df = df.loc[df['INCWAGE'] > 0] # No wage/salary last year
df = df.loc[(df['WKSWORK1'] > 0) & (df['WKSWORK1'] <= 52)] # No weeks worked last year
df = df.loc[(df['UHRSWORKLY'] > 0) & (df['UHRSWORKLY'] < 999)] # No usual hours/week worked last year

In [8]:
print(df.shape)

(85644, 18)


In [9]:
# Restrict to individuals aged 18-64
df = df.loc[(df['AGE'] >=18) & (df['AGE'] <=64)]

In [10]:
print(df.shape)

(78644, 18)


In [11]:
# Restrict to non-Hispanics
df = df.loc[df['HISPAN'] == 0]

In [12]:
print(df.shape)

(63125, 18)


In [13]:
# Restrict to non-mixed-race Blacks and Whites
df = df.loc[df.RACE.isin([100, 200])]
df['blk'] = np.where(df['RACE'] == 200, 1, 0)

In [14]:
print(df.shape)

(55508, 19)


In [15]:
# Restrict to adult civilians
df = df.loc[df['POPSTAT'] == 1]

In [16]:
print(df.shape)

(55069, 19)


### Engineer features

In [17]:
# Flag for part-time usual work
df['pt'] = np.where(df['UHRSWORKLY'] < 35, 1, 0)

In [18]:
# Non-linearities for age
df['AGE2'] = df['AGE'] ** 2
df['AGE3'] = df['AGE'] ** 3

In [19]:
# Ensure education coded correctly
df = df.loc[df['EDUC'] <= 125] # 999 is missing

In [20]:
# Hourly wage
df['hrwage'] = df['INCWAGE'] / (df['WKSWORK1'] * df['UHRSWORKLY'])

In [21]:
df['hrwage'].describe()

count    55069.000000
mean        31.269112
std        184.416973
min          0.000962
25%         14.375000
50%         21.634615
75%         34.188034
max      25481.250000
Name: hrwage, dtype: float64

In [22]:
# Restrict to people earning 1 < hr_wage < 100
df = df.loc[(df['hrwage'] > 1) & (df['hrwage'] < 100)]

In [23]:
print(df['hrwage'].describe())

count    53671.000000
mean        25.907302
std         17.049376
min          1.016667
25%         14.202864
50%         21.634615
75%         33.333333
max         99.759615
Name: hrwage, dtype: float64


In [24]:
# Log hourly wage
df['lnwage'] = np.log(df['hrwage'])

In [25]:
df['INCWAGE'].describe()

count     53671.000000
mean      52682.839876
std       41242.872191
min          25.000000
25%       24617.500000
50%       43680.000000
75%       70000.000000
max      420000.000000
Name: INCWAGE, dtype: float64

In [26]:
# Flag for whether total earnings > 50,000 or not
df['50k'] = np.where(df['INCWAGE'] > 50000, 1, 0)

In [27]:
df = df[['lnwage', 'hrwage', '50k', 'pt', 'INCWAGE', 'WKSWORK1', 'UHRSWORKLY', 
         'AGE', 'AGE2', 'AGE3', 'SEX', 'blk', 'MARST', 'SCHLCOLL', 'EDUC']]

In [28]:
print(df.head())

     lnwage     hrwage  50k  pt  INCWAGE  WKSWORK1  UHRSWORKLY  AGE  AGE2  \
0  2.445686  11.538462    0   1  18000.0        52          30   21   441   
2  1.657229   5.244755    0   0  12000.0        52          44   61  3721   
4  2.445686  11.538462    0   1  12000.0        52          20   37  1369   
6  3.179655  24.038462    1   0  55000.0        52          44   53  2809   
8  3.442059  31.251250    1   0  50002.0        40          40   62  3844   

     AGE3  SEX  blk  MARST  SCHLCOLL  EDUC  
0    9261    1    0      6       5.0    60  
2  226981    2    0      6       0.0    73  
4   50653    1    0      6       5.0    73  
6  148877    2    0      4       5.0    73  
8  238328    1    0      1       0.0   111  


In [29]:
categorical_features = ['SEX', 'MARST', 'SCHLCOLL', 'EDUC', 'pt', 'blk']
for col in categorical_features:
    df[col] = df[col].astype('category')

In [30]:
df.dtypes

lnwage         float64
hrwage         float64
50k              int32
pt            category
INCWAGE        float64
WKSWORK1         int64
UHRSWORKLY       int64
AGE              int64
AGE2             int64
AGE3             int64
SEX           category
blk           category
MARST         category
SCHLCOLL      category
EDUC          category
dtype: object

# Descriptive differences in earnings by race 

### Association between race and likelihood of annual earnings surpassing 50K (i.e. "high-income")

In [31]:
np.corrcoef(df['50k'], df['blk'])

array([[ 1.        , -0.10890816],
       [-0.10890816,  1.        ]])

As compared to being white, being black is associatd with a lower likelihood of being high-income.

In [32]:
print(f"Percent of high-income blacks: {round(df[df.blk==1]['50k'].mean()*100, 2)}%")
print(f"Percent of high-income whites: {round(df[df.blk==0]['50k'].mean()*100, 2)}%")
print(f"Difference: {round((df[df.blk==0]['50k'].mean() - df[df.blk==1]['50k'].mean())*100, 2)}%")

Percent of high-income blacks: 27.9%
Percent of high-income whites: 42.76%
Difference: 14.86%


#### Linear probability model

In [96]:
y = df['50k'].reset_index(drop=True)
predictors = df[['blk', 'AGE', 'AGE2', 'AGE3', 'pt']].reset_index(drop=True)
predictors['blk'] = predictors['blk'].astype(int)
predictors['pt'] = predictors['pt'].astype(int)
for col in ['SEX', 'MARST', 'SCHLCOLL', 'EDUC']:
    onehot_vector = pd.DataFrame(OneHotEncoder().fit_transform(df[[col]]).toarray()[:, 1:])
    onehot_vector.columns = [str(i)+'_'+col[:3] for i in onehot_vector.columns]
    predictors = pd.concat([predictors, onehot_vector], axis=1)
predictors = sm.add_constant(predictors) # add constant

In [97]:
OLS(y, predictors, hasconst=True).fit(cov_type='HC3', use_t=True).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    50k   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     1259.
Date:                Fri, 26 Mar 2021   Prob (F-statistic):               0.00
Time:                        09:51:56   Log-Likelihood:                -28954.
No. Observations:               53671   AIC:                         5.797e+04
Df Residuals:                   53639   BIC:                         5.826e+04
Df Model:                          31                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3332      0.080      4.148      0.000       0.176       0.491
blk           -0.0733      0.005    -14.478      0.000      -0.083      -0.063
AGE           -0.0193      0.005     -3.989      0.000      -0.029      -0.010
AGE2           0.0009      0.000      6.780      0.000       0.001       0.001
AGE3       -8.663e-06   1.08e-06     -8.042      0.000   -1.08e-05   -6.55e-06
pt            -0.2688      0.004    -66.484      0.000      -0.277      -0.261
0_SEX         -0.1804      0.004    -49.260      0.000      -0.188      -0.173
0_MAR         -0.0522      0.017     -2.995      0.003      -0.086      -0.018
1_MAR         -0.0931      0.013     -6.923      0.000      -0.119      -0.067
2_MAR         -0.0616      0.006     -9.560      0.000      -0.074      -0.049
3_MAR         -0.0793      0.016     -4.995      0.000      -0.110      -0.048
4_MAR         -0.0910      0.005    -17.442      0.000      -0.101      -0.081
0_SCH          0.2098      0.018     11.639      0.000       0.174       0.245
1_SCH          0.0819      0.045      1.810      0.070      -0.007       0.171
2_SCH         -0.0258      0.013     -1.949      0.051      -0.052       0.000
3_SCH         -0.0316      0.016     -1.936      0.053      -0.064       0.000
4_SCH          0.0021      0.011      0.193      0.847      -0.019       0.023
0_EDU          0.1145      0.102      1.122      0.262      -0.086       0.315
1_EDU         -0.0084      0.082     -0.103      0.918      -0.170       0.153
2_EDU         -0.0116      0.060     -0.195      0.846      -0.128       0.105
3_EDU         -0.1063      0.055     -1.929      0.054      -0.214       0.002
4_EDU         -0.0216      0.054     -0.400      0.689      -0.128       0.084
5_EDU         -0.0028      0.053     -0.052      0.958      -0.107       0.101
6_EDU         -0.0227      0.054     -0.421      0.674      -0.128       0.083
7_EDU          0.0583      0.052      1.128      0.259      -0.043       0.160
8_EDU          0.1411      0.052      2.726      0.006       0.040       0.243
9_EDU          0.1733      0.052      3.316      0.001       0.071       0.276
10_EDU         0.1885      0.052      3.618      0.000       0.086       0.291
11_EDU         0.3491      0.052      6.744      0.000       0.248       0.451
12_EDU         0.4788      0.052      9.220      0.000       0.377       0.581
13_EDU         0.5770      0.053     10.830      0.000       0.473       0.681
14_EDU         0.5637      0.053     10.658      0.000       0.460       0.667
==============================================================================
Omnibus:                    19550.706   Durbin-Watson:                   1.892
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2717.034
Skew:                           0.112   Prob(JB):                         0.00
Ku

Holding other factors in the model constant, Blacks have about an 7.3% lower probability of earning >$50k per year

### Association between race and log hourly wage

In [33]:
np.corrcoef(df['lnwage'], df['blk'])

array([[ 1.        , -0.11854933],
       [-0.11854933,  1.        ]])

Being black is also correlated with lower hourly wages

In [34]:
print(f"Mean hourly wage, Blacks: ${round(df[df.blk==1]['hrwage'].mean(), 2)}")
print(f"Mean hourly wage, Whites: ${round(df[df.blk==0]['hrwage'].mean(), 2)}")
print(f"Difference: ${round((df[df.blk==0]['hrwage'].mean() - df[df.blk==1]['hrwage'].mean()), 2)}")

Mean hourly wage, Blacks: $21.34
Mean hourly wage, Whites: $26.73
Difference: $5.39


#### OLS regression

In [98]:
y = df['lnwage'].reset_index(drop=True)
OLS(y, predictors, hasconst=True).fit(cov_type='HC3', use_t=True).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 lnwage   R-squared:                       0.296
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     665.6
Date:                Fri, 26 Mar 2021   Prob (F-statistic):               0.00
Time:                        09:52:30   Log-Likelihood:                -45032.
No. Observations:               53671   AIC:                         9.013e+04
Df Residuals:                   53639   BIC:                         9.041e+04
Df Model:                          31                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7981      0.142     12.662      0.000       1.520       2.076
blk           -0.1094      0.007    -15.589      0.000      -0.123      -0.096
AGE            0.0582      0.008      7.478      0.000       0.043       0.073
AGE2          -0.0008      0.000     -3.974      0.000      -0.001      -0.000
AGE3        2.664e-06   1.62e-06      1.640      0.101   -5.19e-07    5.85e-06
pt            -0.1937      0.009    -22.032      0.000      -0.211      -0.176
0_SEX         -0.2118      0.005    -42.615      0.000      -0.222      -0.202
0_MAR         -0.1029      0.024     -4.304      0.000      -0.150      -0.056
1_MAR         -0.1771      0.018     -9.582      0.000      -0.213      -0.141
2_MAR         -0.0999      0.008    -12.037      0.000      -0.116      -0.084
3_MAR         -0.1464      0.023     -6.490      0.000      -0.191      -0.102
4_MAR         -0.1334      0.007    -19.561      0.000      -0.147      -0.120
0_SCH          0.0819      0.040      2.027      0.043       0.003       0.161
1_SCH          0.0736      0.135      0.546      0.585      -0.190       0.338
2_SCH         -0.0823      0.021     -4.003      0.000      -0.123      -0.042
3_SCH         -0.0635      0.023     -2.796      0.005      -0.108      -0.019
4_SCH         -0.0218      0.014     -1.537      0.124      -0.050       0.006
0_EDU         -0.0803      0.144     -0.556      0.578      -0.363       0.203
1_EDU         -0.0342      0.133     -0.257      0.797      -0.295       0.226
2_EDU         -0.1126      0.110     -1.023      0.306      -0.329       0.103
3_EDU         -0.2498      0.104     -2.391      0.017      -0.454      -0.045
4_EDU         -0.1348      0.101     -1.331      0.183      -0.333       0.064
5_EDU         -0.0778      0.099     -0.783      0.433      -0.273       0.117
6_EDU         -0.0978      0.101     -0.964      0.335      -0.297       0.101
7_EDU          0.0561      0.097      0.579      0.563      -0.134       0.246
8_EDU          0.1750      0.097      1.803      0.071      -0.015       0.365
9_EDU          0.2232      0.097      2.291      0.022       0.032       0.414
10_EDU         0.2505      0.097      2.574      0.010       0.060       0.441
11_EDU         0.4950      0.097      5.102      0.000       0.305       0.685
12_EDU         0.6449      0.097      6.638      0.000       0.454       0.835
13_EDU         0.8200      0.099      8.261      0.000       0.625       1.015
14_EDU         0.8030      0.099      8.143      0.000       0.610       0.996
==============================================================================
Omnibus:                     4391.268   Durbin-Watson:                   1.852
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10292.104
Skew:                          -0.509   Prob(JB):                         0.00
Ku

Holding other factors in the model constant, Blacks earn about 10.4% lower hourly wages (0.104 = 1-exp(-0.1094))

# Discrimination in predicting high/low earners

### Train-test split

In [35]:
y = df['50k']
X = df[['blk', 'AGE', 'AGE2', 'AGE3', 'SEX', 'EDUC', 'SCHLCOLL', 'MARST', 'pt']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=999)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=999)

In [36]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(38642, 9)
(9661, 9)
(5368, 9)


### Discriminatory model: Includes race

In [37]:
model1 = lgb.LGBMClassifier(objective='binary',
                           random_state=999,
                           metric='logloss')

In [38]:
model1.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='logloss', early_stopping_rounds=10, verbose=False)

LGBMClassifier(metric='logloss', objective='binary', random_state=999)

In [39]:
disc_pred = model1.predict(X_test)

In [40]:
metrics(y_test, disc_pred)

{'accuracy': 0.7547,
 'sensitivity/recall': 0.7206,
 'specificity': 0.7773,
 'precision': 0.6827,
 'f1': 0.7011,
 'roc_auc': 0.749}

In [41]:
discrim = combine_prediction(X_test, disc_pred)

In [42]:
print("Model performance: Blacks")
mask = discrim[discrim.blk==1]
metrics(mask['50k'], mask['pred'])

Model performance: Blacks


{'accuracy': 0.7799,
 'sensitivity/recall': 0.5244,
 'specificity': 0.874,
 'precision': 0.6051,
 'f1': 0.5619,
 'roc_auc': 0.6992}

In [43]:
print("Model performance: Whites")
mask = discrim[discrim.blk==0]
metrics(mask['50k'], mask['pred'])

Model performance: Whites


{'accuracy': 0.75,
 'sensitivity/recall': 0.7436,
 'specificity': 0.7547,
 'precision': 0.69,
 'f1': 0.7158,
 'roc_auc': 0.7492}

In [44]:
print(f"Percent of Blacks predicted to be high-income: {round(discrim[discrim.blk==1]['pred'].mean()*100, 2)}%")
print(f"Percent of Whites predicted to be high-income: {round(discrim[discrim.blk==0]['pred'].mean()*100, 2)}%")
print(f"Difference: {round((discrim[discrim.blk==0]['50k'].mean() - discrim[discrim.blk==1]['50k'].mean())*100, 2)}%")

Percent of Blacks predicted to be high-income: 23.33%
Percent of Whites predicted to be high-income: 45.63%
Difference: 15.43%


### Naive model: Excluding race

In [45]:
model2 = lgb.LGBMClassifier(objective='binary',
                           random_state=999,
                           metric='logloss')

In [46]:
X_train_mod = X_train.loc[:, X_train.columns != 'blk']
X_val_mod = X_val.loc[:, X_val.columns != 'blk']
X_test_mod = X_test.loc[:, X_test.columns != 'blk']
model2.fit(X_train_mod, y_train, eval_set=[(X_val_mod, y_val)], eval_metric='logloss', early_stopping_rounds=10, verbose=False)

LGBMClassifier(metric='logloss', objective='binary', random_state=999)

In [47]:
naive_pred = model2.predict(X_test_mod)

In [48]:
metrics(y_test, naive_pred)

{'accuracy': 0.7493,
 'sensitivity/recall': 0.6884,
 'specificity': 0.7897,
 'precision': 0.6852,
 'f1': 0.6868,
 'roc_auc': 0.7391}

In [49]:
naive = combine_prediction(X_test, naive_pred)

In [50]:
print("Model performance: Blacks")
mask = naive[naive.blk==1]
metrics(mask['50k'], mask['pred'])

Model performance: Blacks


{'accuracy': 0.7715,
 'sensitivity/recall': 0.6756,
 'specificity': 0.8069,
 'precision': 0.563,
 'f1': 0.6142,
 'roc_auc': 0.7412}

In [51]:
print("Model performance: Whites")
mask = naive[naive.blk==0]
metrics(mask['50k'], mask['pred'])

Model performance: Whites


{'accuracy': 0.7451,
 'sensitivity/recall': 0.6899,
 'specificity': 0.7857,
 'precision': 0.7028,
 'f1': 0.6963,
 'roc_auc': 0.7378}

In [52]:
print(f"Percent of Blacks predicted to be high-income: {round(naive[naive.blk==1]['pred'].mean()*100, 2)}%")
print(f"Percent of Whites predicted to be high-income: {round(naive[naive.blk==0]['pred'].mean()*100, 2)}%")
print(f"Difference: {round((naive[naive.blk==0]['50k'].mean() - naive[naive.blk==1]['50k'].mean())*100, 2)}%")

Percent of Blacks predicted to be high-income: 32.3%
Percent of Whites predicted to be high-income: 41.57%
Difference: 15.43%


In [53]:
y = df['50k']
X_reg = df[['blk', 'AGE', 'AGE2', 'AGE3', 'SEX', 'MARST', 'pt', 'EDUC', 'SCHLCOLL']]
for x in X_reg.columns:
    X_reg[x] = X_reg[x].astype(float)
#X_reg = df[['blk', 'AGE', 'AGE2', 'AGE3', 'SEX', 'MARST', 'pt']]
results = OLS(y, X_reg,hasconst=False).fit()
print(results.params)
print(results.tvalues)

blk        -0.074327
AGE        -0.014878
AGE2        0.000765
AGE3       -0.000008
SEX        -0.181746
MARST      -0.018293
pt         -0.261424
EDUC        0.007673
SCHLCOLL   -0.003221
dtype: float64
blk        -14.458406
AGE        -13.127495
AGE2        19.673391
AGE3       -19.238039
SEX        -49.256033
MARST      -19.718031
pt         -49.979573
EDUC        82.793255
SCHLCOLL    -1.780773
dtype: float64


c:\users\jking\documents\pythonvirtualenv\ml_discrimination\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
